In [1]:
import os
import pickle as pkl
import numpy as np
import render_model
from smpl.smpl_webuser.serialization import load_model
import cv2
#import pyvista as pv
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


import matplotlib.pyplot as plt


    


In [2]:
predicted_poses = np.load(r'/mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D//Alphapose//AlphaPose//examples//res//test_set//downtown_arguing_00//MotionBert//person_1//all_thetas.npy')
predicted_translation = np.load(r'/mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D//Alphapose//AlphaPose//examples//res//test_set//downtown_arguing_00//MotionBert//person_1//all_translations.npy')

def renderImage(model,img_path,camPose,camIntrinsics):
    
    print(img_path)
    img = cv2.imread(img_path)
    class cam:
        pass
    cam.rt = cv2.Rodrigues(camPose[0:3,0:3])[0].ravel()
    cam.t = camPose[0:3,3]
    cam.f = np.array([camIntrinsics[0,0],camIntrinsics[1,1]])
    cam.c = camIntrinsics[0:2,2]
    h = int(2*cam.c[1])
    w = int(2*cam.c[0])
    im = (render_model.render_model(model, model.f, w, h, cam, img= img)* 255.).astype('uint8')

    return im



In [3]:
seq_name = 'downtown_arguing_00'
datasetDir = r'/mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/3DPW'
file = os.path.join(datasetDir, 'sequenceFiles', 'sequenceFiles', 'test', seq_name + '.pkl')
seq = pkl.load(open(file,'rb'), encoding='latin1')

In [4]:
len(seq["jointPositions"][0][0])

72

In [5]:
predicted_poses = np.load(r'/mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D//Alphapose//AlphaPose//examples//res//test_set//downtown_bus_00//MotionBert//person_1//all_thetas.npy')
predicted_translation = np.load(r'/mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D//Alphapose//AlphaPose//examples//res//test_set//downtown_bus_00//MotionBert//person_1//all_translations.npy')

In [7]:
### MAIN ONE wit hthe difference that not only frames with seq['campose_valid'] = 0 are taken
## Change iModel idx



# Base directory paths
dataset_dir = r'/mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/3DPW'
base_pred_dir = r'/mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/Alphapose/AlphaPose/examples/res/test_set'

# Load a sequence file and its corresponding pose predictions
def process_sequence(seq_name):
    print(f"Processing sequence: {seq_name}")
    
    # Load sequence data
    seq_file = os.path.join(dataset_dir, 'sequenceFiles', 'sequenceFiles', 'test', seq_name + '.pkl')
    seq = pkl.load(open(seq_file, 'rb'), encoding='latin1')

    # Iterate over person folders in the sequence
    seq_pred_dir = os.path.join(base_pred_dir, seq_name, 'MotionBert')
    person_folders = [f for f in os.listdir(seq_pred_dir) if f.startswith("person_")]

    for person_folder in person_folders:
        person_number = int(person_folder[-1])
        person_path = os.path.join(seq_pred_dir, person_folder)
        pose_file = os.path.join(person_path, 'all_thetas.npy')
        trans_file = os.path.join(person_path, 'all_translations.npy')
        
        # Check if the pose file exists
        if not os.path.isfile(pose_file):
            print(f"Pose file not found: {pose_file}")
            continue

        # Load pose predictions
        predicted_poses = np.load(pose_file)
        predicted_translation = np.load(trans_file)
        print(f"Loaded poses for {person_folder}, shape: {predicted_poses.shape}")

        # Load the model (assuming only one model is needed per person)
        if person_number == 1:
            iModel = 0  # Assuming one model for simplicity
        elif person_number == 2:
            iModel = 1
        else: raise ValueError

        if seq['genders'][iModel] == 'm':
            model_path = "/mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/3DPW/readme/smpl/models/basicmodel_m_lbs_10_207_0_v1.0.0.pkl"
        else:
            model_path = "/mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/3DPW/readme/smpl/models/basicModel_f_lbs_10_207_0_v1.0.0.pkl"
        
        model = load_model(model_path)
        model.betas[:10] = np.zeros(10)  # Use zero betas or customize if needed
        
        # Calculate vertices for all frames
        vertices_all = []
        for iFrame in range(len(seq['campose_valid'][iModel])):
            if seq['campose_valid'][iModel][iFrame]:
                model.pose[:] = predicted_poses[iFrame]
                model.trans[:] = predicted_translation[iFrame]
                vertices = np.asarray(model)
                vertices_all.append(vertices)

        # Save the vertices for the person
        vertices_all = np.array(vertices_all)  # Convert to numpy array
        output_file = os.path.join(person_path, 'vertices_all_frames.npy')
        np.save(output_file, vertices_all)
        print(f"Saved vertices to {output_file}, shape: {vertices_all.shape}")

# Main processing function
def process_all_sequences():
    # List all sequence folders
    sequence_folders = [f for f in os.listdir(base_pred_dir) if os.path.isdir(os.path.join(base_pred_dir, f))]
    
    for seq_name in sequence_folders:
        process_sequence(seq_name)

# Run the processing
process_all_sequences()

Processing sequence: downtown_arguing_00
1
Loaded poses for person_1, shape: (898, 72)
Saved vertices to /mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/Alphapose/AlphaPose/examples/res/test_set/downtown_arguing_00/MotionBert/person_1/vertices_all_frames.npy, shape: (898, 6890, 3)
2
Loaded poses for person_2, shape: (898, 72)
Saved vertices to /mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/Alphapose/AlphaPose/examples/res/test_set/downtown_arguing_00/MotionBert/person_2/vertices_all_frames.npy, shape: (895, 6890, 3)
Processing sequence: downtown_bar_00
1
Loaded poses for person_1, shape: (1403, 72)
Saved vertices to /mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/Alphapose/AlphaPose/examples/res/test_set/downtown_bar_00/MotionBert/person_1/vertices_all_frames.npy, shape: (1252, 6890, 3)
2
Loaded poses for person_2, shape: (1403, 72)
Saved vertices to /mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/Alphapose/AlphaPose/examples/res/test_set/downtown_bar_00/MotionBert/person_2/ve

In [ ]:
### MAIN ONE
## Change iModel idx



# Base directory paths
dataset_dir = r'/mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/3DPW'
base_pred_dir = r'/mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/Alphapose/AlphaPose/examples/res/test_set'

# Load a sequence file and its corresponding pose predictions
def process_sequence(seq_name):
    print(f"Processing sequence: {seq_name}")
    
    # Load sequence data
    seq_file = os.path.join(dataset_dir, 'sequenceFiles', 'sequenceFiles', 'test', seq_name + '.pkl')
    seq = pkl.load(open(seq_file, 'rb'), encoding='latin1')

    # Iterate over person folders in the sequence
    seq_pred_dir = os.path.join(base_pred_dir, seq_name, 'MotionBert')
    person_folders = [f for f in os.listdir(seq_pred_dir) if f.startswith("person_")]

    for person_folder in person_folders:
        person_path = os.path.join(seq_pred_dir, person_folder)
        pose_file = os.path.join(person_path, 'all_thetas.npy')
        trans_file = os.path.join(person_path, 'all_translations.npy')
        
        # Check if the pose file exists
        if not os.path.isfile(pose_file):
            print(f"Pose file not found: {pose_file}")
            continue

        # Load pose predictions
        predicted_poses = np.load(pose_file)
        predicted_translation = np.load(trans_file)
        print(f"Loaded poses for {person_folder}, shape: {predicted_poses.shape}")

        # Load the model (assuming only one model is needed per person)
        iModel = 0  # Assuming one model for simplicity
        if seq['genders'][iModel] == 'm':
            model_path = "/mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/3DPW/readme/smpl/models/basicmodel_m_lbs_10_207_0_v1.0.0.pkl"
        else:
            model_path = "/mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/3DPW/readme/smpl/models/basicModel_f_lbs_10_207_0_v1.0.0.pkl"
        
        model = load_model(model_path)
        model.betas[:10] = np.zeros(10)  # Use zero betas or customize if needed
        
        # Calculate vertices for all frames
        vertices_all = []
        for iFrame in range(len(seq['campose_valid'][iModel])):
            if seq['campose_valid'][iModel][iFrame]:
                model.pose[:] = predicted_poses[iFrame]
                model.trans[:] = predicted_translation[iFrame]
                vertices = np.asarray(model)
                vertices_all.append(vertices)

        # Save the vertices for the person
        vertices_all = np.array(vertices_all)  # Convert to numpy array
        output_file = os.path.join(person_path, 'vertices_all_frames.npy')
        np.save(output_file, vertices_all)
        print(f"Saved vertices to {output_file}, shape: {vertices_all.shape}")

# Main processing function
def process_all_sequences():
    # List all sequence folders
    sequence_folders = [f for f in os.listdir(base_pred_dir) if os.path.isdir(os.path.join(base_pred_dir, f))]
    
    for seq_name in sequence_folders:
        process_sequence(seq_name)

# Run the processing
process_all_sequences()

Processing sequence: downtown_arguing_00
Loaded poses for person_1, shape: (898, 72)
Saved vertices to /mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/Alphapose/AlphaPose/examples/res/test_set/downtown_arguing_00/MotionBert/person_1/vertices_all_frames.npy, shape: (898, 6890, 3)
Loaded poses for person_2, shape: (898, 72)
Saved vertices to /mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/Alphapose/AlphaPose/examples/res/test_set/downtown_arguing_00/MotionBert/person_2/vertices_all_frames.npy, shape: (898, 6890, 3)
Processing sequence: downtown_bar_00
Loaded poses for person_1, shape: (1403, 72)
Saved vertices to /mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/Alphapose/AlphaPose/examples/res/test_set/downtown_bar_00/MotionBert/person_1/vertices_all_frames.npy, shape: (1252, 6890, 3)
Loaded poses for person_2, shape: (1403, 72)
Saved vertices to /mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/Alphapose/AlphaPose/examples/res/test_set/downtown_bar_00/MotionBert/person_2/vertices_a

In [7]:
# # Sequence and dataset paths
# seq_name = 'downtown_arguing_00'
# datasetDir = r'/mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/3DPW'
# file = os.path.join(datasetDir, 'sequenceFiles', 'sequenceFiles', 'test', seq_name + '.pkl')
# seq = pkl.load(open(file, 'rb'), encoding='latin1')

# # Load models based on gender
# models = []
# for iModel in range(len(seq['v_template_clothed'])):
#     if seq['genders'][iModel] == 'm':
#         model_path = "/mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/3DPW/readme/smpl/models/basicmodel_m_lbs_10_207_0_v1.0.0.pkl"
#     else:
#         model_path = "/mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/3DPW/readme/smpl/models/basicModel_f_lbs_10_207_0_v1.0.0.pkl"

#     model = load_model(model_path)
#     model.betas[:10] = np.zeros(10)  # Set betas to zeros or use provided betas
#     models.append(model)

# # Process all frames
# vertices_all = []  # To store vertices for all frames
# for iFrame in range(len(seq['campose_valid'][0])):
#     iModel = 0  # Assuming one model for simplicity; adjust if multiple models are required
#     if seq['campose_valid'][iModel][iFrame]:
#         # Update model parameters for this frame
#         models[iModel].pose[:] = predicted_poses[iFrame]
#         models[iModel].trans[:] = [0, 0, 0]

#         # Calculate vertices for this frame
#         vertices = np.asarray(models[iModel])
#         vertices_all.append(vertices)

# # Convert to numpy array for easy saving/usage
# vertices_all = np.array(vertices_all)  # Shape: (num_frames, num_vertices, 3)
# print("Vertices array shape (all frames):", vertices_all.shape)

# # Save the vertices to a file
# np.save("vertices_all_frames.npy", vertices_all)

In [8]:
# if __name__ == '__main__':
#     seq_name = 'downtown_arguing_00'
#     datasetDir = r'/mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/3DPW'
#     file = os.path.join(datasetDir, 'sequenceFiles', 'sequenceFiles', 'test', seq_name + '.pkl')
#     seq = pkl.load(open(file,'rb'), encoding='latin1')

#     models = list()
#     for iModel in range(0,len(seq['v_template_clothed'])):
#         if seq['genders'][iModel] == 'm':
#             model = load_model(os.path.join("/mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/3DPW/readme/smpl/models/basicmodel_m_lbs_10_207_0_v1.0.0.pkl"))
#         else:
#             model = load_model(os.path.join("/mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/3DPW/readme/smpl/models/basicModel_f_lbs_10_207_0_v1.0.0.pkl"))

#         model.betas[:10] = seq['betas'][iModel][:10]
#         model.betas[:10] = np.zeros(10)
#         models.append(model)

#     iModel = 0
#     iFrame = 0
#     if seq['campose_valid'][iModel][iFrame]:
        
#         #gt_pose = seq['poses'][iModel][iFrame]
#         #gt_pose = models[iModel].pose[:]
#         models[iModel].pose[:] = predicted_poses[iFrame]
#         #pred_pose = models[iModel].pose[:]

#         #models[iModel].pose[:] = seq['poses'][iModel][iFrame]
#         #models[iModel].trans[:] = seq['trans'][iModel][iFrame]
#         models[iModel].trans[:] = [0,0,0]

        

    
#         print((models[iModel].pose[:]).shape)
#         img_path = os.path.join(datasetDir,'imageFiles', 'imageFiles',seq['sequence']+'/image_{:05d}.jpg'.format(iFrame))
#         # print(type(models[iModel]))
#         vertices = (np.asarray(models[iModel]))
#         print("Vertices shape is:")
#         print(vertices.shape)
#         np.save("vertices_backpack_frame_00_pred.npy", vertices)




#         im = renderImage(models[iModel],img_path,seq['cam_poses'][iFrame],seq['cam_intrinsics'])

        
#         cv2.imwrite('output.png', im)

#         cv2.waitKey(0)
#         cv2.destroyAllWindows()

In [9]:
# Managed visualization. Try to either restrict on pos without translation or find out a way to get the correct translation
#for that it has to be scaled and offset
#use --pixel flag in Motionbert (?)
#Ask motionbert how to evaluate MPJPE

### VISUALIZATION

In [10]:
predicted_poses = np.load(r'/mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D//Alphapose//AlphaPose//examples//res//test_set//downtown_stairs_00//MotionBert//person_1//all_thetas.npy')
predicted_translation = np.load(r'/mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D//Alphapose//AlphaPose//examples//res//test_set//downtown_stairs_00//MotionBert//person_1//all_translations.npy')

In [11]:
if __name__ == '__main__':
    seq_name = 'downtown_stairs_00'
    datasetDir = r'/mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/3DPW'
    file = os.path.join(datasetDir, 'sequenceFiles', 'sequenceFiles', 'test', seq_name + '.pkl')
    seq = pkl.load(open(file,'rb'), encoding='latin1')

    models = list()
    for iModel in range(0,len(seq['v_template_clothed'])):
        if seq['genders'][iModel] == 'm':
            model = load_model(os.path.join("/mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/3DPW/readme/smpl/models/basicmodel_m_lbs_10_207_0_v1.0.0.pkl"))
        else:
            model = load_model(os.path.join("/mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/3DPW/readme/smpl/models/basicModel_f_lbs_10_207_0_v1.0.0.pkl"))

        model.betas[:10] = seq['betas'][iModel][:10]
        models.append(model)

    iModel = 0
    iFrame = 0
    if seq['campose_valid'][iModel][iFrame]:
        
        # gt_pose = seq['poses'][iModel][iFrame]
        # #gt_pose = models[iModel].pose[:]
        models[iModel].pose[:] = predicted_poses[iFrame]
        # pred_pose = models[iModel].pose[:]
        # models[iModel].trans[:] = seq['trans'][iModel][iFrame]

        #models[iModel].trans[:] = predicted_translation[iFrame] # prediction trans

        #models[iModel].trans[:] = seq['trans'][iModel][iFrame] # actual trans

        # #pred_pose[0:3] = gt_pose[0:3]
        

    
        # print((models[iModel].pose[:]).shape)
        # img_path = os.path.join(datasetDir,'imageFiles', 'imageFiles',seq['sequence']+'/image_{:05d}.jpg'.format(iFrame))
        # # print(type(models[iModel]))
        # vertices = (np.asarray(models[iModel]))

        # print(seq['cam_poses'].shape)




        # im = renderImage(models[iModel],img_path,seq['cam_poses'][iFrame],seq['cam_intrinsics'])
        # print("im is type:")
        # print(im.shape)
        
        # cv2.imwrite('output.png', im)

        # cv2.waitKey(0)
        # cv2.destroyAllWindows()

        # Video settings
        video_path = "output_video.avi"
        frame_width, frame_height = 1920, 1080  # Adjust as needed
        fps = 30  # Frames per second
        fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Codec for AVI
        video_writer = cv2.VideoWriter(video_path, fourcc, fps, (frame_width, frame_height))

        # Loop over frames
        for iFrame in range(len(seq['campose_valid'][iModel])):  # Range of frames
            if seq['campose_valid'][iModel][iFrame]:
                models[iModel].pose[:] = predicted_poses[iFrame]
                #pred_pose = models[iModel].pose[:]
                models[iModel].trans[:] = seq['trans'][iModel][iFrame]
                models[iModel].trans[:] = predicted_translation[iFrame]

                #models[iModel].trans[:] = [0,0,0]
                

                img_path = os.path.join(datasetDir, 'imageFiles', 'imageFiles',
                                        seq['sequence'], f'image_{iFrame:05d}.jpg')

                im = renderImage(models[iModel], img_path, seq['cam_poses'][iFrame], seq['cam_intrinsics'])
                resized_im = cv2.resize(im, (frame_width, frame_height))  # Ensure frame size matches video settings
                video_writer.write(resized_im)  # Add frame to video

        video_writer.release()  # Finalize and save video
        print(f"Video saved to {video_path}")

/mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/3DPW/imageFiles/imageFiles/downtown_stairs_00/image_00000.jpg
/mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/3DPW/imageFiles/imageFiles/downtown_stairs_00/image_00001.jpg
/mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/3DPW/imageFiles/imageFiles/downtown_stairs_00/image_00002.jpg
/mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/3DPW/imageFiles/imageFiles/downtown_stairs_00/image_00003.jpg
/mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/3DPW/imageFiles/imageFiles/downtown_stairs_00/image_00004.jpg
/mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/3DPW/imageFiles/imageFiles/downtown_stairs_00/image_00005.jpg
/mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/3DPW/imageFiles/imageFiles/downtown_stairs_00/image_00006.jpg
/mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/3DPW/imageFiles/imageFiles/downtown_stairs_00/image_00007.jpg
/mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/3DPW/imageFiles/imageFiles/downtown_stairs_00/image_00

In [ ]:
# TODO: In motionbert use "in pixel or camera pixel flag an look at output"

In [15]:
predicted_vertices = np.load(r"/mnt/e/Users/Philipp/Dokumente/Pose_Estimation_3D/Alphapose/AlphaPose/examples/res/test_set/downtown_bus_00/MotionBert/person_1/vertices_all_frames.npy")

In [16]:
predicted_vertices.shape

(1066, 6890, 3)